Загрузка модели случайного леса и scaler

In [1]:
from joblib import load
from random import seed
seed(52)

model = load('RFR_model.joblib')
model_bbbp = load('RFR_model_bbbp.joblib')
scaler = load('scaler.joblib')

In [2]:
from rdkit.Chem import QED, rdMolDescriptors, MolToSmiles, MolFromSmiles, Draw, \
                       Descriptors, Lipinski, Crippen, GraphDescriptors
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams
from rdkit.Contrib.SA_Score import sascorer
from rdkit import rdBase
import numpy as np
import warnings

warnings.filterwarnings("ignore")
rdBase.DisableLog('rdApp.error')

def pIC50_score(smile):
    exactmw = Descriptors.ExactMolWt(smile)
    FractionCSP3 = Lipinski.FractionCSP3(smile)
    NumRings = rdMolDescriptors.CalcNumRings(smile)
    NumAromaticRings = Lipinski.NumAromaticRings(smile)
    NumAliphaticRings = Lipinski.NumAliphaticRings(smile)
    NumHeterocycles = rdMolDescriptors.CalcNumHeterocycles(smile)
    NumAromaticHeterocycles = rdMolDescriptors.CalcNumAromaticHeterocycles(smile)
    NumSaturatedHeterocycles = rdMolDescriptors.CalcNumSaturatedHeterocycles(smile)
    NumAliphaticHeterocycles = rdMolDescriptors.CalcNumAliphaticHeterocycles(smile)
    NumSpiroAtoms = rdMolDescriptors.CalcNumSpiroAtoms(smile)
    NumBridgeheadAtoms = rdMolDescriptors.CalcNumBridgeheadAtoms(smile)
    NumAtomStereoCenters = rdMolDescriptors.CalcNumAtomStereoCenters(smile)
    NumUnspecifiedAtomStereoCenters = rdMolDescriptors.CalcNumUnspecifiedAtomStereoCenters(smile)
    CrippenClogP = Crippen.MolLogP(smile)
    hallKierAlpha = GraphDescriptors.HallKierAlpha(smile)

    mol = np.array([[exactmw, FractionCSP3, NumRings, NumAromaticRings, NumAliphaticRings, NumHeterocycles,
                     NumAromaticHeterocycles, NumSaturatedHeterocycles, NumAliphaticHeterocycles,
                     NumSpiroAtoms, NumBridgeheadAtoms, NumAtomStereoCenters, NumUnspecifiedAtomStereoCenters,
                     CrippenClogP, hallKierAlpha]])
    mol = scaler.transform(mol)
    pIC50 = model.predict(mol)[0]
    return pIC50

def bbbp_score(smile):
    exactmw = Descriptors.ExactMolWt(smile)
    FractionCSP3 = Lipinski.FractionCSP3(smile)
    NumRings = rdMolDescriptors.CalcNumRings(smile)
    NumAromaticRings = Lipinski.NumAromaticRings(smile)
    NumAliphaticRings = Lipinski.NumAliphaticRings(smile)
    NumHeterocycles = rdMolDescriptors.CalcNumHeterocycles(smile)
    NumAromaticHeterocycles = rdMolDescriptors.CalcNumAromaticHeterocycles(smile)
    NumSaturatedHeterocycles = rdMolDescriptors.CalcNumSaturatedHeterocycles(smile)
    NumAliphaticHeterocycles = rdMolDescriptors.CalcNumAliphaticHeterocycles(smile)
    NumSpiroAtoms = rdMolDescriptors.CalcNumSpiroAtoms(smile)
    NumBridgeheadAtoms = rdMolDescriptors.CalcNumBridgeheadAtoms(smile)
    NumAtomStereoCenters = rdMolDescriptors.CalcNumAtomStereoCenters(smile)
    NumUnspecifiedAtomStereoCenters = rdMolDescriptors.CalcNumUnspecifiedAtomStereoCenters(smile)
    CrippenClogP = Crippen.MolLogP(smile)
    hallKierAlpha = GraphDescriptors.HallKierAlpha(smile)
    mol = np.array([[exactmw, FractionCSP3, NumRings, NumAromaticRings, NumAliphaticRings, NumHeterocycles,
                     NumAromaticHeterocycles, NumSaturatedHeterocycles, NumAliphaticHeterocycles,
                     NumSpiroAtoms, NumBridgeheadAtoms, NumAtomStereoCenters, NumUnspecifiedAtomStereoCenters,
                     CrippenClogP, hallKierAlpha]])
    bbbp = model_bbbp.predict(mol)[0]
    return bbbp

def QED_score(smile):
    return QED.qed(smile)

def SA_score(smile):
    return sascorer.calculateScore(smile)

def toxicphore(smile):
    params = FilterCatalogParams()
    params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
    catalog = FilterCatalog(params)
    toxic = catalog.HasMatch(smile)
    return toxic

def lipinski_rules(smile):
    num = 0
    if Descriptors.MolWt(smile) > 500:
        num += 1
    if Crippen.MolLogP(smile) > 5:
        num += 1
    if Lipinski.NumHDonors(smile) > 5:
        num += 1
    if Lipinski.NumHAcceptors(smile) > 10:
        num += 1
    return num <= 1

def scores(smile):
    pIC50 = pIC50_score(smile)
    QED = QED_score(smile)
    toxic = toxicphore(smile)
    SA = SA_score(smile)
    lipinski = lipinski_rules(smile)
    bbbp = bbbp_score(smile)
    return f"{pIC50} {bbbp} {QED} {SA} {toxic} {lipinski} ---"

def all_scores(smile):
    pIC50 = pIC50_score(smile)
    QED = QED_score(smile)
    toxic = toxicphore(smile)
    SA = SA_score(smile)
    lipinski = lipinski_rules(smile)
    bbbp = bbbp_score(smile)
    if pIC50 > 6.0:
        if QED > 0.5 and 2.0 < SA < 6.0 and not toxic and lipinski and bbbp:
            return True
    return False

smile = MolFromSmiles('CC(C)(C)c1cc(C(=O)CCCC2CC2)cc2c1OCCC2(C)C')
print(scores(smile))
print(all_scores(smile))

<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for class boost::shared_ptr<class RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.


4.85124783478307 1 0.6377780199717005 2.6749215613864674 False True ---
False


In [9]:
from rdkit.Chem import Descriptors, rdFMCS
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
import random

def scaffold_hopping(reference_smiles, num_variants=5):
    """Генерация аналогов на основе известного лиганда"""
    ref_mol = MolFromSmiles(reference_smiles)
    scaffold = MurckoScaffold.GetScaffoldForMol(ref_mol)
    
    # Генерация вариаций
    variants = []
    for _ in range(num_variants):
        # Создание модифицированного каркаса
        new_mol = Chem.RWMol(scaffold)
        
        # Случайные модификации
        for _ in range(random.randint(3, 10)):
            atom_idx = random.choice(range(new_mol.GetNumAtoms()))
            new_mol.ReplaceAtom(atom_idx, Chem.Atom(random.choice(["N","O","S","F","H"])))
        
        variants.append(Chem.MolToSmiles(new_mol))
    
    return variants

# Пример использования
df = pd.read_csv("dataset_filtered.csv")

new_ligands = []
for _ in range(100):
    known_ligand = random.choice(df["Smiles"])
    new_ligand = scaffold_hopping(known_ligand, 200)
    new_ligands.extend(new_ligand)
#print("Аналоги лиганда:", new_ligands)

In [10]:
results = []

for smile in new_ligands:
    mol = MolFromSmiles(smile)
    if mol is not None:
        if all_scores(mol):
            row = {
                'SMILES': smile,
                'pIC50': pIC50_score(mol),
                'QED': QED_score(mol),
                'SA_Score': SA_score(mol),
                'Toxicophore': toxicphore(mol),
                'Lipinski_violations': lipinski_rules(mol),
                'BBBP': bbbp_score(mol)
            }
            results.append(row)
   

df_result = pd.DataFrame(results)
df_result = df_result[['SMILES', 'pIC50', 'QED', 'SA_Score', 'Toxicophore', 'Lipinski_violations', "BBBP"]]

In [11]:
df.drop_duplicates()
df_result.to_csv('selected_hits.csv', index=False)